In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import json
from shapely.geometry import shape, Point, Polygon

In [2]:
properties = pd.read_csv('residential.csv')
properties.head(10)

,results_key,zpid,street,city,zipcode,latitude,longitude,zestimate,valueChange,low_estimate,...,assessment,yearBuilt,lotSize,houseSize,bathrooms,bedrooms,lastSold,lastSoldPrice,grid_num,updated
0,1,3510018,7042 N Ozark Ave,Chicago,60631,42.009037,-87.820381,582730.0,-22871.0,553594.0,...,544800.0,1990.0,6600.0,3000.0,1.5,4.0,NaN,NaN,NaN,2019-04-09 14:01:47
1,2,3511171,6947 N Olcott Ave,Chicago,60631,42.007469,-87.813278,416296.0,-310.0,395481.0,...,387950.0,1923.0,7350.0,1757.0,1.5,3.0,1995-09-20,94500.0,NaN,2019-04-09 14:01:47
2,3,3496766,7420 N Oriole Ave,Chicago,60631,42.015762,-87.816704,488252.0,-1793.0,434544.0,...,325320.0,1953.0,3750.0,2600.0,3.0,4.0,2016-04-27,530000.0,NaN,2019-04-09 14:01:47
3,4,3496779,7401 N Ottawa Ave,Chicago,60631,42.015228,-87.817223,540485.0,15323.0,513461.0,...,452730.0,1957.0,5246.0,2819.0,2.5,4.0,NaN,NaN,NaN,2019-04-09 14:01:47
4,5,3509965,7121 N Ozanam Ave,Chicago,60631,42.009953,-87.820915,419367.0,7971.0,398399.0,...,323570.0,1935.0,5941.0,1567.0,2.5,3.0,2017-05-09,467000.0,NaN,2019-04-09 14:01:47
5,6,3510161,7115 N Ottawa Ave,Chicago,60631,42.010014,-87.817238,386591.0,-5760.0,363396.0,...,339690.0,1958.0,6636.0,1222.0,1.5,3.0,2000-10-24,225000.0,NaN,2019-04-09 14:01:47
6,7,3496778,7405 N Ottawa Ave,Chicago,60631,42.015347,-87.817223,338458.0,2185.0,321535.0,...,297070.0,1957.0,5500.0,1157.0,1.0,3.0,NaN,NaN,NaN,2019-04-09 14:01:47
7,8,3497240,7508 N Osceola Ave,Chicago,60631,42.017311,-87.813026,284740.0,5647.0,270503.0,...,237110.0,1950.0,3750.0,917.0,2.0,2.0,1993-11-02,84666.0,NaN,2019-04-09 14:01:47
8,9,3512515,6587 N Northwest Hwy APT 201,Chicago,60631,42.000824,-87.812477,168532.0,1176.0,160105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-09 14:06:51
9,10,3512510,6587 N Northwest Hwy APT 2,Chicago,60631,42.000824,-87.812477,178532.0,1218.0,167820.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-09 14:06:51


In [3]:
properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132016 entries, 0 to 132015
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   results_key                132016 non-null  int64  
 1   zpid                       132016 non-null  int64  
 2   street                     132016 non-null  object 
 3   city                       131996 non-null  object 
 4   zipcode                    132016 non-null  int64  
 5   latitude                   132016 non-null  float64
 6   longitude                  132016 non-null  float64
 7   zestimate                  132016 non-null  float64
 8   valueChange                115060 non-null  float64
 9   low_estimate               132016 non-null  float64
 10  high_estimate              132016 non-null  float64
 11  neighborhood               132003 non-null  object 
 12  neighborhood_zindex_value  125850 non-null  float64
 13  hometype                   11

In [5]:
properties['unit_zestimate'] = properties.zestimate / properties.houseSize
properties['unit_high_est'] = properties.high_estimate / properties.houseSize
properties['unit_low_est'] = properties.low_estimate / properties.houseSize

In [7]:
with open('neighorhood_polys.json','r') as f:
    neighborhoods = json.load(f)

In [86]:
positions = pd.DataFrame(zip(properties.longitude, properties.latitude))

In [65]:
def point_lookup(polygon_dict, point):
    
    for i in range(len(list(polygon_dict.keys()))):
        result = Polygon(polygon_dict[list(polygon_dict.keys())[i]]).contains(point) # zero index is there because of zoning type in zoning data
        
        if result == True:
            return list(polygon_dict.keys())[i]
            break 

In [89]:
properties['true_neighborhood'] = np.nan
for i in range(len(properties)):
    properties['true_neighborhood'][i] = point_lookup(neighborhoods, Point(positions.iloc[i,:].values))

/Users/mark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mark/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [94]:
properties.true_neighborhood.value_counts(dropna=False)

West Ridge          6213
Mount Greenwood     5849
Dunning             5653
Norwood Park        5651
Lake View           4308
                    ... 
Magnificent Mile       8
Greektown              5
Grant Park             4
Millenium Park         1
East Side              1
Name: true_neighborhood, Length: 96, dtype: int64

In [99]:
unit_value_by_neighborhood = properties.groupby('true_neighborhood')\
                             [['unit_zestimate', 'unit_high_est', 'unit_low_est']].mean()

In [104]:
unit_value_by_neighborhood

,unit_zestimate,unit_high_est,unit_low_est
true_neighborhood,,,
Albany Park,1508.167781,2050.207016,1155.972745
Andersonville,2003.563160,3107.395104,1467.692676
Archer Heights,839.866143,1078.521795,600.642888
Armour Square,264.469474,284.875728,245.080884
Ashburn,153.938723,162.913007,144.477890
...,...,...,...
West Ridge,4243.048035,5011.548172,3713.274794
West Town,5338.776832,5652.563969,5048.727047
Wicker Park,412.878641,474.839395,370.849164


In [103]:
unit_value_by_neighborhood[unit_value_by_neighborhood.unit_zestimate.isna()]

,unit_zestimate,unit_high_est,unit_low_est
true_neighborhood,,,
Greektown,NaN,NaN,NaN
Millenium Park,NaN,NaN,NaN
